# Derinlemesine k-Means Kümeleme

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling
import numpy as np

**k-means** algoritması, etiketlenmemiş çok boyutlu bir veri kümesi içinde önceden belirlenmiş sayıda kümeyi arar. Bunu, optimal kümelemenin nasıl göründüğüne dair basit bir anlayış kullanarak başarır:

- "Küme merkezi (cluster center)", kümeye ait tüm noktaların aritmetik ortalamasıdır.
- Her nokta kendi küme merkezine diğer küme merkezlerinden daha yakındır.

Bu iki varsayım, **k-means** modelinin temelidir.

İlk olarak, dört farklı blob içeren iki boyutlu bir veri kümesi oluşturalım.
Bunun denetimsiz bir algoritma olduğunu vurgulamak için etiketleri görselleştirmenin dışında bırakacağız.


In [ ]:
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)

plt.scatter(X[:, 0], X[:, 1], s=50);

Gözle, dört kümeyi seçmek nispeten kolaydır.
**k-means** algoritması bunu otomatik olarak yapar ve Scikit-Learn'de tipik bir tahmin API'sini kullanır:


In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)
kmeans.fit(X)

y_kmeans = kmeans.predict(X)

Bu etiketlerin renklendirdiği verileri çizerek sonuçları görselleştirelim.
k-means tahmincisi tarafından belirlenen küme merkezlerini de çizdirelim.

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

K-means algoritması (en azından bu basit durumda) noktaları, onları gözle nasıl atayabileceğimize çok benzer şekilde kümelere atar.

Ancak bu algoritmanın bu kümeleri nasıl bu kadar çabuk bulduğunu merak edebilirsiniz! Sonuçta, küme atamalarının olası kombinasyonlarının sayısı, veri noktalarının sayısında üsteldir. Bu da kapsamlı ve çok maliyetli bir aramaya sebep olacaktır.

Neyse ki bizim için böyle kapsamlı bir arama gerekli değildir: bunun yerine, k-means **beklenti–maksimizasyon (expectation–maximization)** olarak bilinen sezgisel yinelemeli bir yaklaşımı içerir.


## k-Means Algoritması: Beklenti-Maksimizasyon Yaklaşımı (Expectation–Maximization)

K-means kümeleme yaklaşımındaki beklenti-maksimizasyon yaklaşımı (E-M) aşağıdaki adımlardan oluşur:

1. Bazı küme merkezlerini tahmin edin
2. Birleşene kadar tekrarlayın
   1. **E-Step:** noktaları en yakın küme merkezine atayın
   2. **M-Step:** küme merkezlerini ortalamaya (means) ayarla

Burada "E-step" veya "Beklenti (Expectation) adımı", her noktanın ait olduğu kümeye ilişkin beklentimizi güncellemeyi içerdiğinden böyle adlandırılmıştır.

"M adımı" veya "Maksimizasyon adımı", küme merkezlerinin konumunu tanımlayan bazı uygunluk fonksiyonunun maksimize edilmesini içerdiği için bu şekilde adlandırılır. (K-means'de maksimizasyon, her kümedeki verilerin basit bir ortalamasını alarak gerçekleştirilir.)


![(run code in Appendix to generate image)](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.11-expectation-maximization.png?raw=1)


### Beklenti-maksimizasyon ile ilgili bazı uyarılar

Beklenti-maksimizasyon algoritmasını kullanırken dikkat edilmesi gereken birkaç konu vardır.

#### Global optimal sonuç elde edilemeyebilir
E–M prosedürü her adımda sonucu iyileştireceği garanti etse de, bunun global optimal bir çözüme yol açacağının garantisi yoktur.

#### Küme sayısı önceden seçilmelidir
k-means ile ilgili diğer zorluk, başlangıçta kaç küme beklediğinizi söylemeniz gerektiğidir: **kümelerin sayısını verilerden öğrenemez.**
Örneğin, algoritmadan altı küme tanımlamasını istersek, mutlu bir şekilde ilerleyecek ve en iyi altı kümeyi bulacaktır:

In [ ]:
labels = KMeans(6, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

Sonucun anlamlı olup olmadığı kesin olarak cevaplanması zor bir sorudur; bu noktada sezgisel bir yaklaşım olan [siluet analizi (Silhouette Analysis)](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html) analizi kullanılır.

Alternatif olarak, küme sayısı başına uygunluğun daha iyi nicel ölçümüne sahip Gaussian Mixture Models gibi daha karmaşık bir kümeleme algoritması kullanılabilir veya uygun sayıda kümeyi seçebilen DBSCAN benzeri yaklaşımlar da kullanılabilir.


#### k-means doğrusal küme sınırları ile sınırlıdır

k-means'in temel model varsayımları (noktalar kendi küme merkezlerine diğerlerinden daha yakın olacaktır), kümeler karmaşık geometrilere sahipse algoritmanın genellikle etkisiz olacağı anlamına gelir.

**Özellikle, k-means kümeleri arasındaki sınırlar her zaman doğrusal olacaktır, bu da daha karmaşık sınırlar için başarısız olacağı anlamına gelir.**


In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(200, noise=.05, random_state=0)

In [ ]:
labels = KMeans(2, random_state=0).fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

Bu durum, verileri doğrusal bir ayırmanın mümkün olduğu daha yüksek bir boyuta yansıtmak için bir çekirdek dönüşümü kullandığımız [Depth: Support Vector Machines](05.07-Support-Vector-Machines.ipynb) 'deki tartışmayı hatırlatıyor.

Aynı hileyi k-means'in doğrusal olmayan sınırları keşfetmesine izin vermek için kullanmayı hayal edebiliriz.

Bu çekirdekleştirilmiş k-means'ın bir versiyonu **Scikit-Learn'de "SpectralClustering"** tahmincisi içinde uygulanmaktadır.


In [ ]:
from sklearn.cluster import SpectralClustering

model = SpectralClustering(n_clusters=2, affinity='nearest_neighbors',
                           assign_labels='kmeans')

labels = model.fit_predict(X)

plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

Bu çekirdek dönüşümü yaklaşımıyla, çekirdekleştirilmiş k-means'in kümeler arasındaki daha karmaşık doğrusal olmayan sınırları bulabildiğini görüyoruz.


#### k-means çok sayıda örnek için yavaş olabilir!

k-means'in her yinelemesinin veri kümesindeki her noktaya erişmesi gerektiğinden, örnek sayısı arttıkça algoritma nispeten yavaş olabilir.

**Her yinelemede tüm verileri kullanma gereksiniminin azaltılıp azaltılmayacağını merak edebilirsiniz; örneğin, her adımda küme merkezlerini güncellemek için verilerin bir alt kümesini kullanmak bir yaklaşım olabilir.**

Bu, bir biçimi "sklearn.cluster.MiniBatchKMeans" içinde uygulanan batch-based k-means algoritmasının arkasındaki fikirdir.


## Örnekler

Algoritmanın bu sınırlamalarına dikkat ederek, k-means'i çok farklı durumlarda avantajımıza kullanabiliriz.

Şimdi birkaç örneğe göz atacağız.

### Example 1: Rakamlar üzerinde K-means

K-means kullanarak benzer rakamları tanımlamaya çalışacağız (orijinal etiket bilgisini kullanmadan).

Rakamları yükleyerek ve ardından ``KMeans`` kümelerini bularak başlayacağız.
Rakamların 64 özelliğe sahip 1.797 örnekten oluştuğunu ve 64 özelliğin her birinin 8×8 görüntüdeki bir pikselin parlaklığıdır:


In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Kümeleme daha önce yaptığımız gibi gerçekleştirilebilir:

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits.data)
kmeans.cluster_centers_.shape

Sonuç 64 boyutta 10 kümedir.
Küme merkezlerinin kendilerinin 64 boyutlu noktalar olduğuna ve küme içindeki "tipik" sayılar olarak yorumlanabileceklerine dikkat edin.

Bu küme merkezlerinin neye benzediğini görelim:

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))

centers = kmeans.cluster_centers_.reshape(10, 8, 8)

for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

**Etiketler olmadan bile,** ``KMeans``in, belki 1 ve 8 hariç, merkezleri tanınabilir rakamlar olan kümeleri bulabildiğini görüyoruz.

k-means, kümenin kimliği hakkında hiçbir şey bilmediğinden, 0-9 etiketlerine izin verilebilir. Bunu, öğrenilen her küme etiketini, içinde bulunan gerçek etiketlerle eşleştirerek düzeltebiliriz:


In [ ]:
from scipy.stats import mode

labels = np.zeros_like(clusters)
print("labels", labels)
for i in range(10):
    mask = (clusters == i)
    print("mask",mask)
    print("mask len",len(mask))
    print(digits.target[mask])
    labels[mask] = mode(digits.target[mask])[0]

Artık denetimsiz kümelememizin veriler içinde benzer rakamları bulmada ne kadar doğru olduğunu kontrol edebiliriz:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(digits.target, labels)

Yalnızca basit bir *k*-means algoritmasıyla, giriş basamaklarının %80'i için doğru gruplamayı keşfettik!
Bunun için karışıklık matrisini kontrol edelim:

In [ ]:
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(digits.target, labels)

sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label');

Daha önce görselleştirdiğimiz küme merkezlerinden beklediğimiz gibi, asıl karışıklık noktası sekizler ve birler arasındadır.

Ancak bu yine de k-means'i kullanarak, bilinen herhangi bir etikete  başvurmadan bir rakam sınıflandırıcıyı oluşturabileceğimizi gösteriyor!

Sadece eğlence için, hadi bunu daha da ileri götürmeye çalışalım.
k-means gerçekleştirmeden önce verileri önceden işlemek için t-SNE algoritmasını kullanabiliriz .

t-SNE, kümeler içindeki noktaları korumada özellikle usta olan, doğrusal olmayan bir gömme algoritmasıdır.

Nasıl olduğunu görelim:


In [ ]:
from sklearn.manifold import TSNE

# Project the data: this step will take several seconds
tsne = TSNE(n_components=2, init='random', random_state=0)
digits_proj = tsne.fit_transform(digits.data)

# Compute the clusters
kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits_proj)

# Permute the labels
labels = np.zeros_like(clusters)
for i in range(10):
    mask = (clusters == i)
    labels[mask] = mode(digits.target[mask])[0]

# Compute the accuracy
accuracy_score(digits.target, labels)

Bu sonuç, etiketleri kullanmadan yaklaşık %92 sınıflandırma doğruluğu elde ettiğimizi göstermektedir.
Bu, dikkatli kullanıldığında denetimsiz öğrenmenin gücünü net bir şekilde göstermektedir. Veri kümesinden elle veya gözle yapılması zor olabilecek bilgileri çıkarabildiğini görmüş olduk.

### Example 2: Renk sıkıştırması için K-means

Kümelemenin ilginç bir uygulaması, resimlerdeki renk sıkıştırmasıdır.
Örneğin, milyonlarca renge sahip bir görüntünüz olduğunu hayal edin.
Çoğu görüntüde, çok sayıda renk kullanılmayacak ve görüntüdeki piksellerin çoğu benzer veya hatta aynı renklere sahip olacaktır.

Örneğin, Scikit-Learn ``datasets`` modülünden gelen aşağıdaki şekilde gösterilen resmi düşünün.

In [ ]:
# Note: this requires the ``pillow`` package to be installed
from sklearn.datasets import load_sample_image
china = load_sample_image("china.jpg")
ax = plt.axes(xticks=[], yticks=[])
ax.imshow(china);

Görüntünün kendisi, 0'dan 255'e kadar tamsayılar olarak kırmızı/mavi/yeşil katkıları içeren "(yükseklik, genişlik, RGB)" boyutunda üç boyutlu bir dizide saklanır:

In [ ]:
china.shape

Bu piksel kümesini görmemizin bir yolu, üç boyutlu bir renk uzayında bir nokta bulutu gibidir.
Verileri ``[n_samples x n_features]`` şeklinde yeniden şekillendireceğiz ve renkleri 0 ile 1 arasında olacak şekilde yeniden ölçeklendireceğiz.

In [ ]:
data = china / 255.0 # use 0...1 scale
data = data.reshape(427 * 640, 3)
data.shape

Verimlilik için 10.000 piksellik bir alt küme kullanarak bu pikselleri bu renk uzayında görselleştirebiliriz:

In [ ]:
def plot_pixels(data, title, colors=None, N=10000):
    if colors is None:
        colors = data
    
    # choose a random subset
    rng = np.random.RandomState(0)
    i = rng.permutation(data.shape[0])[:N]
    colors = colors[i]
    R, G, B = data[i].T
    
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].scatter(R, G, color=colors, marker='.')
    ax[0].set(xlabel='Red', ylabel='Green', xlim=(0, 1), ylim=(0, 1))

    ax[1].scatter(R, B, color=colors, marker='.')
    ax[1].set(xlabel='Red', ylabel='Blue', xlim=(0, 1), ylim=(0, 1))

    fig.suptitle(title, size=20);

In [ ]:
plot_pixels(data, title='Girdi renk alanı: 16 milyon olası renk')

Şimdi piksel alanı boyunca bir k-means kümeleme kullanarak bu 16 milyon rengi yalnızca 16 renge indirgeyelim.

Çok büyük bir veri kümesiyle uğraştığımız için, sonucu standart k-means algoritmasından çok daha hızlı hesaplamak için verilerin alt kümeleri üzerinde çalışan mini-batch k-means'i kullanacağız:


In [ ]:
import warnings
warnings.simplefilter('ignore')  # Fix NumPy issues.

from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(16)
kmeans.fit(data)
new_colors = kmeans.cluster_centers_[kmeans.predict(data)]

plot_pixels(data, colors=new_colors,
            title="Azaltılmış renk alanı: 16 renk")

Sonuç, her piksele en yakın küme merkezinin renginin atandığı orijinal piksellerin yeniden renklendirilmesidir.


In [ ]:
china_recolored = new_colors.reshape(china.shape)

fig, ax = plt.subplots(1, 2, figsize=(16, 6),
                       subplot_kw=dict(xticks=[], yticks=[]))
fig.subplots_adjust(wspace=0.05)

ax[0].imshow(china)
ax[0].set_title('Original Image', size=16)

ax[1].imshow(china_recolored)
ax[1].set_title('16-color Image', size=16);

Sağdaki görselde bazı ayrıntılar kesinlikle kayboluyor, ancak genel görüntü hala kolayca tanınabilir.

Sağdaki bu görüntü, yaklaşık 1 milyonluk bir sıkıştırma faktörüne ulaşıyor!

Bu k-means'in ilginç bir uygulaması olsa da, görüntülerdeki bilgileri sıkıştırmanın kesinlikle daha iyi yolları var.
